In [ ]:
library(tidyverse)
library(tidymodels)

# Project Final Report: Do Age and Hours Played Affect if a Player is Subscribed

## **Introduction:**

A group in CS at UBC is interested in understanding how people play video games. They've set up a server running MineCraft and are recording play sessions. Running the server is not easy: need to have the right hardware resources, software licenses, recruiting efforts, etc. Our task is to formulate and answer a predictive question about the data. Present the full analysis, from reading the data to communicating results. 

The question we our trying to asnwer is: Can the number of hours played and the age of players accurately predict whether or not a given player is subscribed to the newsletter in this dataset?

**Data Description:** In the players dataset read from players.csv, there are 196 observations each comprised of 7 variables. Eash observation represents an individual player from the Minecraft server.
- The first variable titled experience is a character variable showing the playing experience of each player like amateur, beginner, regular, veteran, and pro.
- The next variable titled subscribe is a logical variable showing whether or not each player has subscribed to a certain game-related newsletter.
- The next variable titled hashedEmail is a character variable showing the hashed format of each player's email address.
- The next variable titled played_hours is a double quantitative variable showing how many hours each players played the game for. The average number of hours played is 5.85 spanning a range from 0.0 to 223.1 hours.
- The next variable titled name is a character variable containing the first name of each player.
- The next variable titled gender is a character variable showing the preffered gender of each player not just limited to Male, Female, and Other.
- The last variable titled Age is a double quantitative variable showing the age of each player. The average age is 20.52 years old spanning a range from 8 to 50 years old.
- 
Some potential issues with the data may stem from the fact that the hashedEmails are coded in a way that they are almost unusable as data, the fact that there are many players who played the game for a recorded 0.0 hours, the fact that the gender variable seems to have been customizeable by the players meaning that any value could be put into the variable, and the fact that there are some values of NA for the Age variable.

## **Methods and Plan:**

I will be using the k-nearest neighbors classification method to find out whether or not time played and age can predict if a player has subscribed to the newsletter. I am choosing to use the k-nearest neighbors classification method because the predictors are two quantitative variables trying to predict a categorical variable so the variables fit the model. A potential weakness in the method may be that the distribution of time played is so right-skewed with so many values at or around 0 that it may cause some error in the classification model. I will center and scale both the played_hours and age variables before inputting them into the model, after which I will split the data into training and testing groups with a 75/25 split. Afterwards, a validation set will be created and cross-validation will be used to find the best fitting model to the data.

**Questions:**

Broad: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

Specific: Can the number of hours played and the age of players accurately predict whether or not a given player is subscribed to the newsletter in this dataset?

I will use the k-nearest neighbors classification model using the hours_played and Age variables to predict the outcome of the subscribe variable. Using the two numerical variables to predict a categorical variable will accurately fit the outline of the k-nearest neighbors prediction method.

**Exploratory Data Analysis and Visualization:**

Reading in and tidying the players dataset:

In [ ]:
library(tidyverse)
players <- read_csv("https://raw.githubusercontent.com/brendancfellows/project_planning/refs/heads/main/players.csv")
head(players)


Table of mean values for each quantitative variable:

In [ ]:
tidy_players <- players |>
    rename(hashed_email = hashedEmail, age = Age) |>
    mutate(subscribe = as.factor(subscribe)) |>
    filter(!is.na(age))
head(tidy_players)

In [ ]:
means <- tidy_players |>
    summarize(played_hours = mean(played_hours), age = mean(age, na.rm = TRUE))
means

Exploratory visualizations:

In [ ]:
hours_vs_age <- tidy_players |>
    ggplot(aes(x = age, y = played_hours)) +
    geom_point() +
    labs(x = "Age of Player (years)", y = "Time Played (hours)") +
    ggtitle("Time Played (hours) versus Age of Player (years)")
hours_vs_age

age_dist <- tidy_players |>
    ggplot(aes(x = age)) +
    geom_histogram() +
    labs(x = "Age of Player (years)", y = "Count") +
    ggtitle("Distribution of Age of Player (years)")
age_dist

hours_dist <- tidy_players |>
    ggplot(aes(x = played_hours)) +
    geom_histogram() +
    labs(x = "Time Played (hours)", y = "Count") +
    ggtitle("Distribution of Time Played (hours)")
hours_dist

Data Analysis:

In [ ]:
set.seed(3312)

players_split <- initial_split(tidy_players, prop = 0.75, strata = subscribe)
players_training <- training(players_split)
players_testing <- testing(players_split)

players_recipe <- recipe(subscribe ~ age + played_hours, data = players_training) |>
    step_center(all_predictors()) |>
    step_scale(all_predictors())

players_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

k_vals <- tibble(neighbors = seq(from = 1, to = 100, by = 1))

players_vfold <- vfold_cv(players_training, v = 5, strata = subscribe)

players_results <- workflow() |>
    add_recipe(players_recipe) |>
    add_model(players_spec) |>
    tune_grid(resamples = players_vfold, grid = k_vals) |>
    collect_metrics()

accuracies <- players_results |>
    filter(.metric == "accuracy")

head(accuracies)

accuracy_vs_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
    geom_point() + 
    geom_line() + 
    labs(x = "Neighbors", y = "Accuracy Estimate") 
accuracy_vs_k


In [ ]:
set.seed(3312)
best_k <- accuracies |>
    arrange(desc(mean)) |>
    head(1) |>
    pull(neighbors)
best_k

In [ ]:
players_spec_2 <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k) |>
    set_engine("kknn") |>
    set_mode("classification")

knn_fit <- workflow() |>
    add_recipe(players_recipe) |>
    add_model(players_spec_2) |>
    fit(data = players_training) 

players_test_predictions <- predict(knn_fit, players_testing) |>
    bind_cols(players_testing)

players_metrics <- players_test_predictions |>
    metrics(truth = subscribe, estimate = .pred_class) |>
    filter(.metric == "accuracy")
players_metrics 

confusion <- players_test_predictions |>
    conf_mat(truth = subscribe, estimate = .pred_class)
confusion

In [ ]:
age_grid <- seq(min(tidy_players$age),
                max(tidy_players$age),
                length.out = 100)
plh_grid <- seq(min(tidy_players$played_hours),
                max(tidy_players$played_hours),
                length.out = 100)
apgrid <- as_tibble(expand.grid(age = age_grid,
                                played_hours = plh_grid))

knnPredGrid <- predict(knn_fit, apgrid)

prediction_table <- bind_cols(knnPredGrid, apgrid) |>
  rename(subscribe = .pred_class)

wkflw_plot <-
  ggplot() +
  geom_point(data = tidy_players,
             mapping = aes(x = age,
                           y = played_hours,
                           color = subscribe),
             alpha = 0.75) +
  geom_point(data = prediction_table,
             mapping = aes(x = age,
                           y = played_hours,
                           color = subscribe),
             alpha = 0.02,
             size = 5) +
  labs(color = "Subcribed or Not",
       x = "Age",
       y = "Played Hours") +
  scale_color_manual(values = c("darkorange", "steelblue")) +
  theme(text = element_text(size = 12))

wkflw_plot

Insights:

From these visualizations is can be gathered that there is a slight nonlinear relationship between time played and age. Generally, the players with the most time played are from within the 15 to 25 years old age range. However, this correlates with the fact that the distribution of ages follows the same trend with a unimodal shape with most players being aged somewhere between 15 and 25 years old. Also, the distribution of time played is very heavily right-skewed with almost all of the players being around 0 hours played.

## Discussion

Summarize: The older 